In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [102]:
col_titles = ["date", "min", "avg", "max"]
winter_df = pd.DataFrame(columns=col_titles)
summer_df = pd.DataFrame(columns=col_titles)

def getData(df, date):
    """ Takes in a dataframe and the date and inserts the min, avg, and
        max temperatures of that date to the dataframe. """

    url = requests.get("https://www.almanac.com/weather/history/IL/Chicago/" + date)
    content = url.text
    # soup = BeautifulSoup(content, 'html5lib')
    html = pd.read_html(content, header=0)[0]

    html["date"] = date
    data = [html.iloc[0][2], html.iloc[0][1], html.iloc[1][1], html.iloc[2][1]]
    df.loc[len(df)] = data

# Summer range: 1946-06-21 to 2020-09-22
# Winter range: 1945-12-21 to 2020-03-20
years = range(1946,2021)
summer_start = '06-21'
summer_end = '09-22'
winter_start = '12-21'
winter_end = '03-20'
summer_dates = []
winter_dates = []
for y in years:
    ss = str(y) + '-' + summer_start
    se = str(y) + '-' + summer_end
    ws = str(y-1) + '-' + winter_start
    we = str(y) + '-' + winter_end
    summer = pd.date_range(ss, se, freq = 'D')
    for s in summer:
        summer_dates.append(str(s.date()))
    winter = pd.date_range(ws, we, freq = 'D')
    for w in winter:
        winter_dates.append(str(w.date()))




In [98]:
batch = range(0, len(winter_dates), 1000)
for dates in [winter_dates[i : i + 1000] for i in batch]:
    for date in dates:
        getData(winter_df, date)
winter_df.to_csv("winter_data.csv")

In [106]:
batch = range(0, len(summer_dates), 500)

for dates in [summer_dates[i : i + 500] for i in batch]:
    for date in dates:
        getData(summer_df, date)       
summer_df.to_csv("summer_data.csv")


In [31]:
winter_df = pd.read_csv('winter_data.csv', index_col=0)
summer_df = pd.read_csv('summer_data.csv', index_col=0)

In [32]:
# Remove ºF from temperature data
def removeUnit(val):
    ''' Removes the unit from the value. Returns value.'''
    new = str(val)
    new = new[ : new.find(' ')]
    return new

winter_df['min'] = winter_df['min'].apply(removeUnit)
winter_df['avg'] = winter_df['avg'].apply(removeUnit)
winter_df['max'] = winter_df['max'].apply(removeUnit)
summer_df['min'] = summer_df['min'].apply(removeUnit)
summer_df['avg'] = summer_df['avg'].apply(removeUnit)
summer_df['max'] = summer_df['max'].apply(removeUnit)